# Ames Housing Data Preprocessing and Feature Selection

This project aims to predict housing prices in Ames, Iowa based on the famous Ames Iowa Housing Dataset from the Ames City Assessor's Office, made available by Dean De Cock of Truman University in 2011. Professor De Cock's original paper can be found [here.](http://jse.amstat.org/v19n3/decock.pdf)

Presentation Slides for this project can be found [here.](https://www.beautiful.ai/player/-N4J5UYshyuRtwl5G4I7)

This notebook includes python code for data extraction, cleaning and transformation. Model Fitting and Analysis code can be found in the Ames_Models notebook.

# Sections and steps

- <a href="#IMP">Reading in Data</a><br>
- <a href="#PRE">Second Pass - Bivariate Analysis and Statistics</a><br>
    - Pearson's correlation for numerical variables
    - ANOVA and condensing for categorical variables
- <a href="#PRE">Preparing Data for Models</a><br>
    - Linear Models
        -dummify categorical variables
    - Non Linear Models
        -label encode categorical variables
- <a href="#PRE">First Pass Linear Models</a><br>
- <a href="#PRE">Feature Selection</a><br>
- <a href="#EXP">Exporting Cleaned Datasets</a><br>

<p><a name="IMP"></a></p>

## Importing Packages, Reading in Data

In [1]:
#packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.formula.api import ols

#helper module
from helper_module import *

#setting options to view dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#warnings
import warnings
warnings.filterwarnings("ignore")

#helper module functionality
%load_ext autoreload
%autoreload 2

#### Reading in train data

In [2]:
train = pd.read_csv('./data/cleaned/train_c.csv')
train.head()
#train.drop(['Unnamed: 0'], axis=1, inplace=True)
print(f"Read in Ames Housing Train Data - Shape: {train.shape}")

Read in Ames Housing Train Data - Shape: (1451, 69)


In [3]:
train.drop('Unnamed: 0', axis=1, inplace = True)
train.head()

,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,BldgType,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MoSold,SaleType,SaleCondition,SalePrice,log_SalePrice,AgeHome,YrsSnRmdl,TotBthrm,TotFloors,NeighbQual,HasFireplace,HasAlley,HasFence,HasPool,HasGarage,NearNeg,NearPos
0,RL,65,8450,Reg,Lvl,Inside,Gtl,1Fam,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,854,0,1710,3,1,Gd,Typ,NoFireplace,Attchd,RFn,2,TA,TA,Y,0,61,0,0,0,2,WD,Normal,208500,12.247694,5,5,3.5,1,TA,1,0,0,0,1,0,0
1,RL,80,9600,Reg,Lvl,FR2,Gtl,1Fam,6,8,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,0,0,1262,3,1,TA,Typ,TA,Attchd,RFn,2,TA,TA,Y,298,0,0,0,0,5,WD,Normal,181500,12.109011,31,31,2.5,1,Ex,1,0,0,0,1,1,0
2,RL,68,11250,IR1,Lvl,Inside,Gtl,1Fam,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,162,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,866,0,1786,3,1,Gd,Typ,TA,Attchd,RFn,2,TA,TA,Y,0,42,0,0,0,9,WD,Normal,223500,12.317167,7,6,3.5,1,TA,1,0,0,0,1,0,0
3,RL,60,9550,IR1,Lvl,Corner,Gtl,1Fam,7,5,Gable,CompShg,Wd Sdng,Wd Shng,None,0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,756,0,1717,3,1,Gd,Typ,Gd,Detchd,Unf,3,TA,TA,Y,0,35,272,0,0,2,WD,Abnorml,140000,11.849398,91,36,2.0,1,Ex,1,0,0,0,1,0,0
4,RL,84,14260,IR1,Lvl,FR2,Gtl,1Fam,8,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1053,0,2198,4,1,Gd,Typ,TA,Attchd,RFn,3,TA,TA,Y,192,84,0,0,0,12,WD,Normal,250000,12.429216,8,8,3.5,1,Ex,1,0,0,0,1,0,0


In [4]:
train.dtypes

MSZoning          object
LotFrontage        int64
LotArea            int64
LotShape          object
LandContour       object
LotConfig         object
LandSlope         object
BldgType          object
OverallQual        int64
OverallCond        int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea         int64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object
HeatingQC         object
CentralAir        object
Electrical        object
2ndFlrSF           int64
LowQualFinSF       int64
GrLivArea          int64
BedroomAbvGr       int64
KitchenAbvGr       int64
KitchenQual       object
Functional        object
FireplaceQu       object


In [5]:
#making sure the binary variables are categorical
for var in ['HasFireplace', 'HasFence', 'HasGarage', 'NearNeg']:
    train[var]= train[var].astype(str)

<p><a name="PRE"></a></p>

## Preprocessing Data for Models

In [ ]:
cat_var = train.select_dtypes('O').columns.tolist()
num_var = train.select_dtypes('number').columns.tolist()
print(len(num_var),' numerical variables')
print(len(cat_var),' categorical variables')

In [ ]:
SalePrice = train['SalePrice']
log_Price = train['log_SalePrice']

In [ ]:
train.drop('SalePrice', axis = 1, inplace = True)
train.drop('log_SalePrice', axis = 1, inplace = True)

### Linear Models
#### Dummification of Categorical Variables

In [ ]:
preproc = pd.get_dummies(train, columns = cat_var, drop_first=True)

In [ ]:
print(train.shape)
print(preproc.shape)

In [ ]:
preproc.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
preproc, log_Price, test_size = 0.3, random_state = 22)

In [ ]:
X_train.head()

##### Standardizing numerical variables

In [ ]:
nlist = list(train.select_dtypes('number'))
print('Numerical Column Names: \n', nlist)

num_indx = []
for nvar in nlist:
    num_indx.append(train.select_dtypes('number').columns.get_loc(nvar))
    
print('\nNumerical Columns Indices: \n', num_indx)

In [ ]:
num_indx

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

t = [('num', StandardScaler(), num_indx)]
transf = ColumnTransformer(transformers = t,
                          remainder = 'passthrough')

In [ ]:
num_indx

In [ ]:
X_train_scaled = transf.fit_transform(X_train)
X_test_scaled = transf.transform(X_test)

In [ ]:
X_train_scaled.shape

In [ ]:
def look_num(varlist):
    count = 0
    for var in varlist:
        print(var)
        x = X_train_scaled[:,count]
        y = y_train
        print('*'*50)
            
        #info about pearson's correlation
        corr = pearsonr(x, y)
        print('Pearsons correlation (r):', round(corr[0],3))
        count += 1
        print('*'*50)
        
        #Figures
        sns.set_theme(style="whitegrid")
        
        #seaborn Histogram
        f, ax = plt.subplots(1, 2, figsize = (10,6))
        sns.histplot(x=x, data=X_train_scaled, ax = ax[0])
        plt.xticks(rotation = 45)
        
        #seaborn Scatterplot
        sns.scatterplot(x=x, y = y, ax = ax[1])
        plt.xticks(rotation=45)
        
        yield f.show()

In [ ]:
view_num = look_num(nlist)

In [ ]:
next(view_num)

# TESTING


In [ ]:
def get_residuals(model, X, Y):
    
    residuals = Y - model.predict(X)
    
    return residuals

In [ ]:
def model_evaluate(model, X, Y):
    import numpy as np
    from sklearn.metrics import mean_squared_error, r2_score

    rmse = mean_squared_error(Y, 
                            model.predict(X), 
                            squared = False)
    r2 = r2_score(Y, model.predict(X))
    
    if len(np.where(model.predict(X)>20)[0]) > 0: # specific to this dataset
        # Issue, model prediction is too large
        rmse_dollars = np.nan
    else:
        rmse_dollars = mean_squared_error(np.exp(Y), 
                            np.exp(model.predict(X)), 
                            squared = False)

    print('RMSE Log Sale Price: ', rmse)
    print('R^2: ', r2)
    print('RMSE: $', rmse_dollars)
    
    return rmse, r2, rmse_dollars

In [ ]:
def plot_the_model(model, x_train, y_train, x_test, y_test):
    import numpy as np
    from matplotlib import pyplot as plt

    fig, axs = plt.subplots(1,2) 
    fig.set_figheight(5)
    fig.set_figwidth(10)

    x = np.linspace(10, 14.5, 100)
    y = np.linspace(10, 14.5, 100)
    
    # --------------------------------
    # Plot the Actual vs. Predicted
    # --------------------------------
    axs[0].scatter(model.predict(x_train), 
         y_train, color='k')
    axs[0].plot(x,y, 'r-')
 
    axs[0].set_ylabel('Actual log of Sale Price')
    axs[0].set_xlabel('Predicted log of Sale Price')
    axs[0].tick_params(axis='x', rotation=45)
    axs[0].legend(['line','Train'])
    
    axs[1].scatter(model.predict(x_test), 
         y_test, color='teal') 
    axs[1].plot(x,y, 'r-')

    axs[1].set_ylabel('Actual log of Sale Price')
    axs[1].set_xlabel('Predicted log of Sale Price')
    axs[1].tick_params(axis='x', rotation=45)
    axs[1].legend(['line','Test'])

In [ ]:
def get_linear_regress_model(X, Y):
    from sklearn.linear_model import LinearRegression
    lm = LinearRegression()
    
    model = lm.fit(X, Y)
    print('LM model object created.')
    
    return model

In [ ]:
def get_lasso_model(X, Y):
    from sklearn.linear_model import Lasso
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import KFold

    skfold = KFold(n_splits=5, shuffle=True, random_state=0)
    lasso_gs  = Lasso()
    lasso_gs.set_params(normalize=False)

    params = {
        "alpha": [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]
    }

    gs = GridSearchCV(lasso_gs, params, scoring = 'r2', cv=skfold)
    gs.fit(X, Y)
    
    model = gs.best_estimator_
    
    print('Lasso model object created.')
    print(gs.best_estimator_)
    
    return model

In [ ]:
def get_ridge_model(X, Y):
    from sklearn.linear_model import Ridge
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import KFold

    skfold = KFold(n_splits=3, shuffle=True, random_state=0)
    ridge_gs  = Ridge()
    ridge_gs.set_params(normalize=False)

    params = {
        "alpha": [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]
    }

    gs = GridSearchCV(ridge_gs, params, scoring = 'r2', cv=skfold)
    gs.fit(X, Y)
    
    model = gs.best_estimator_
    
    print('Ridge model object created.')
    print(gs.best_estimator_)
    
    return model

In [ ]:
def get_elasticNet_model(X, Y):
    from sklearn.linear_model import ElasticNet
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import KFold

    skfold = KFold(n_splits=3, shuffle=True, random_state=0)
    elastic_gs  = ElasticNet()
    elastic_gs.set_params(normalize=False)

    params = [{
        "alpha": [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01],
        "l1_ratio":[0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90]
    }]

    gs = GridSearchCV(elastic_gs, params, scoring = 'r2', cv=skfold)
    gs.fit(X_train_scaled, y_train)
    
    model = gs.best_estimator_
    
    print('Elastic Net model object created.')
    print(gs.best_estimator_)
    
    return model

In [ ]:
# initializing these variables for model comparision
model_descrp = []
r2_log_train = []
r2_log_test = []
rmse_log_train = []
rmse_log_test = []
rmse_dollars_train = []
rmse_dollars_test = []

In [ ]:
def append_myMetrics(which_model, r2_train, r2_test, rmse_train, rmse_test, rmse_train_dollars, rmse_test_dollars):
    
    r2_log_train_temp = r2_train
    r2_log_test_temp = r2_test
    rmse_log_train_temp = rmse_train
    rmse_log_test_temp = rmse_test
    rmse_dollars_train_temp = rmse_train_dollars
    rmse_dollars_test_temp = rmse_test_dollars

    model_descrp.append(which_model)
    if r2_log_train_temp <= 1 and r2_log_train_temp > 0:
        r2_log_train.append(r2_log_train_temp)
    elif r2_log_train_temp > 1:   
        r2_log_train.append(1000)
    else:
        r2_log_train.append(-1000)

    if r2_log_test_temp <= 1 and r2_log_test_temp > 0:
        r2_log_test.append(r2_log_test_temp)
    elif r2_log_test_temp > 1:   
        r2_log_test.append(1000)
    else:
        r2_log_test.append(-1000)
    
    if rmse_log_train_temp <= 10:
        rmse_log_train.append(rmse_log_train_temp)
    elif rmse_log_train_temp > 10:   
        rmse_log_train.append(1000)

    if rmse_log_test_temp <= 10:
        rmse_log_test.append(rmse_log_test_temp)
    elif rmse_log_test_temp > 10:   
        rmse_log_test.append(1000)
    
    rmse_dollars_train.append(rmse_dollars_train_temp)
    rmse_dollars_test.append(rmse_dollars_test_temp)

In [ ]:
#linear regression model
lm_model = get_linear_regress_model(X_train_scaled, y_train)

In [ ]:
rmse_train, r2_train, rmse_train_dollars= model_evaluate(lm_model, X_train_scaled, y_train)

In [ ]:
rmse_test, r2_test, rmse_test_dollars= model_evaluate(lm_model, X_test_scaled, y_test)

In [ ]:
residuals_train = get_residuals(lm_model, X_train_scaled, y_train)
plt.hist(residuals_train, 100, facecolor = 'teal');
plt.title('Histograme of Residuals - Training Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
residuals_test = get_residuals(lm_model, X_test_scaled, y_test)
plt.hist(residuals_test, 100, facecolor = 'magenta');
plt.title('Histograme of Residuals - Test Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
print('Linear Regression Model - No Regularization');
plot_the_model(lm_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
which_model = 'Linear Regression'
append_myMetrics(which_model, r2_train, r2_test, rmse_train, rmse_test, rmse_train_dollars, rmse_test_dollars)

In [ ]:
#Lasso

In [ ]:
from sklearn.linear_model import Lasso
lasso  = Lasso()
lasso.set_params(normalize=False)
lasso_scores_train = []
lasso_scores_test  = []

alphas = np.linspace(0.0001,0.05,100)
coefs_lasso  = []

for alpha in alphas:
        lasso.set_params(alpha=alpha)
        lasso.fit(X_train_scaled, y_train)  
        coefs_lasso.append(lasso.coef_)
        
        lasso_scores_train.append(lasso.score(X_train_scaled, y_train))
        lasso_scores_test.append(lasso.score(X_test_scaled, y_test))

coefs_lasso = pd.DataFrame(coefs_lasso, index = alphas, columns = preproc.columns)  
# coefs_lasso.head()

In [ ]:
lasso_scores_train = np.array(lasso_scores_train) 
lasso_scores_test  = np.array(lasso_scores_test)

In [ ]:
plt.plot(alphas, lasso_scores_train, label=r'$train\ R^2$')
plt.plot(alphas, lasso_scores_test, label=r'$test\ R^2$')
plt.title(r'Lasso Train-Test $R^2$ Comparison')

lasso_underfit = lasso_scores_train < lasso_scores_test
first_underfit  = np.min(alphas[lasso_underfit])
print('lambda = ', first_underfit)
plt.axvline(first_underfit, linestyle='--', color='g', label='optimal lambda', alpha=0.4)

plt.legend(loc=1)
plt.xlabel(r'hyperparameter $\lambda$')
plt.ylabel(r'$R^2$')

In [ ]:
#Lasso 2.0

In [ ]:
lasso_model = get_lasso_model(X_train_scaled, y_train)

In [ ]:
rmse_train, r2_train, rmse_train_dollars= model_evaluate(lasso_model, X_train_scaled, y_train)

In [ ]:
rmse_test, r2_test, rmse_test_dollars= model_evaluate(lasso_model, X_test_scaled, y_test)

In [ ]:
residuals_train = get_residuals(lasso_model, X_train_scaled, y_train)
plt.hist(residuals_train, 100, facecolor = 'black');
plt.title('Histograme of Residuals - Training Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
residuals_test = get_residuals(lasso_model, X_test_scaled, y_test)
plt.hist(residuals_test, 100, facecolor = 'magenta');
plt.title('Histograme of Residuals - Test Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
print('Linear Regression Model - Lasso');
plot_the_model(lasso_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
which_model = 'Lasso'
append_myMetrics(which_model, r2_train, r2_test, rmse_train, rmse_test, rmse_train_dollars, rmse_test_dollars)

In [ ]:
#Ridge

In [ ]:
ridge_model = get_ridge_model(X_train_scaled, y_train)

In [ ]:
rmse_train, r2_train, rmse_train_dollars= model_evaluate(ridge_model, X_train_scaled, y_train)

In [ ]:
rmse_test, r2_test, rmse_test_dollars= model_evaluate(ridge_model, X_test_scaled, y_test)

In [ ]:
residuals_train = get_residuals(ridge_model, X_train_scaled, y_train)
plt.hist(residuals_train, 100, facecolor = 'black');
plt.title('Histograme of Residuals - Training Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
residuals_test = get_residuals(ridge_model, X_test_scaled, y_test)
plt.hist(residuals_test, 100, facecolor = 'magenta');
plt.title('Histograme of Residuals - Test Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
print('Linear Regression Model - Ridge');
plot_the_model(ridge_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
which_model = 'Ridge'
append_myMetrics(which_model, r2_train, r2_test, rmse_train, rmse_test, rmse_train_dollars, rmse_test_dollars)

In [ ]:
#Elastic Net

In [ ]:
EN_model = get_elasticNet_model(X_train_scaled, y_train)

In [ ]:
rmse_train, r2_train, rmse_train_dollars= model_evaluate(EN_model, X_train_scaled, y_train)

In [ ]:
rmse_test, r2_test, rmse_test_dollars= model_evaluate(EN_model, X_test_scaled, y_test)

In [ ]:
residuals_train = get_residuals(EN_model, X_train_scaled, y_train)
plt.hist(residuals_train, 100, facecolor = 'black');
plt.title('Histograme of Residuals - Training Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
residuals_test = get_residuals(EN_model, X_test_scaled, y_test)
plt.hist(residuals_test, 100, facecolor = 'magenta');
plt.title('Histograme of Residuals - Test Dataset')
plt.ylabel('Counts')
plt.xlabel('Residual')

In [ ]:
print('Linear Regression Model - Elastic Net');
plot_the_model(EN_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
which_model = 'Elastic Net'
append_myMetrics(which_model, r2_train, r2_test, rmse_train, rmse_test, rmse_train_dollars, rmse_test_dollars)

In [ ]:
pd.options.display.float_format = '{:,.4f}'.format
zipped = list(zip(model_descrp,
                 rmse_log_train, rmse_log_test, 
                 rmse_dollars_train, rmse_dollars_test,
                 r2_log_train, r2_log_test))

results = pd.DataFrame(zipped, columns=['model',
                                       'rmse log train',
                                       'rmse log test',
                                       'rmse train',
                                       'rmse test',
                                       'r2 train',
                                       'r2 test'])

#results.head(10)
results_sorted = results.sort_values('rmse log test', ascending = True)
results_sorted.head(10)

In [ ]:
EN_model.get_params

In [ ]:
EN_model.intercept_

In [ ]:
coefs = pd.DataFrame(
    EN_model.coef_,
    columns=["Coefficients"],
    index=preproc.columns,
)
coefs.head()

In [ ]:
coefs.head(25)

### Non Linear Models
#### Label Encoding of Categorical Variables

### All Models
#### Standardization of numerical variables
#### Split train and validate datasets

<p><a name="EXP"></a></p>

## Exporting Cleaned Datasets